In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import os

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Config
## 1.1 Config Environment

In [ ]:
# get root directory
root_dir = str(Path().absolute().parent)

In [ ]:
# load environment variables
from dotenv import load_dotenv
env_path = f"{root_dir}/.env"
load_dotenv(dotenv_path=env_path)

## 1.2 Config LLM and Embedding

In [ ]:
llm = ChatOpenAI(
    model="doubao-1-5-pro-32k-250115",
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_URL"),
    openai_proxy="http://127.0.0.1:7897",
    temperature=0
)

In [ ]:
embedding = OpenAIEmbeddings(
    model=os.environ.get("EMBEDDING_MODEL"),
    api_key=os.environ.get("EMBEDDING_API_KEY"),
    base_url=os.environ.get("EMBEDDING_API_URL"),
    dimensions=os.environ.get("EMBEDDING_DIMENSIONS", None),
    check_embedding_ctx_length=False,
    openai_proxy="http://127.0.0.1:7897",
)

## 1.3 Config input/output file path

In [ ]:
context_file_path = f"{root_dir}/data/input/context-2000.csv"
ap_file_path = f"{root_dir}/data/output-doubao-1-5-pro/attack_prompt-2000.csv"
dataset_file_path = f"{root_dir}/data/output-doubao-1-5-pro/dataset-2000.csv"
response_file_path = f"{root_dir}/data/output-doubao-1-5-pro/attack_prompt_response-2000.csv"

## 1.4 import input context and preprocess it

In [ ]:
contexts = pd.read_csv(context_file_path)
contexts = contexts["context"].tolist()

## 1.5 config chroma vector database

In [ ]:
vectordb = Chroma(
    collection_name="retrieval_database",
    persist_directory=f"{root_dir}/data/database/chroma-doubao-2000/",
    embedding_function=embedding,
)

In [ ]:
# only run this if you want to create a new database
# for i, context in tqdm(enumerate(contexts_splits), total=len(contexts_splits), desc="Adding to vector database"):
#     vectordb.add_texts(texts=[context], embedding=embedding)

In [ ]:
len(vectordb.get()['documents'])

In [ ]:
vectordb.get()

In [ ]:
results = vectordb.similarity_search_with_score(
    query="i am 45 years old and after i eat anything my face goes bright red and my neck and then takes hours to go away it is making me feel very self concious and also if i drink alcohol this happens i am so fed up with this and cant find out what is causing this",
    k=5,
)
print(results)
res = [
    {
        "metadata": doc.metadata,
        "page_content": doc.page_content,
        "score": score
    }
    for doc, score in results  # 遍历 res 中的 (Document, score) 元组
]
print(res)

# 2. Build RAG System

## 2.1 construct rag system graph

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from typing_extensions import TypedDict

In [ ]:
# state class
class State(TypedDict):
    query: str
    contexts: list[str]
    response: str

In [ ]:
# Nodes
def retrieve(state: State):
    query = state["query"]
    contexts = vectordb.similarity_search(query=query, k=5)
    contexts = [context.page_content for context in contexts]
    return {"query": query, "contexts": contexts}

def generate(state: State):
    query = state["query"]
    contexts = state["contexts"]

    contexts = "\n\n".join(context for context in contexts)

    prompt = hub.pull("rlm/rag-prompt")
    rag_chain = prompt | llm | StrOutputParser()

    response = rag_chain.invoke({"question": query, "context": contexts})

    return {"query": query, "contexts": contexts, "response": response}

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(State)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

graph = workflow.compile()

In [ ]:
# test the graph
state = State(
    query="I am a parent concerned about my child's health. I know that my child has been pooing 5 - 6 times a day for a week, which increased to 7 times in the last few days. The poo is watery with green stringy bits and there is a very bad nappy rash. The recovery time is estimated to be 5 - 7 days. Based on this information, please query the diagnosis and any cautions from the RAG system.",
)

graph.invoke(state)['response']

## 2.2 generate response for attack prompts

In [ ]:
df = pd.read_csv(ap_file_path)
contexts = pd.read_csv(context_file_path)
df["context"] = contexts.iloc[:, 0]
df = df[df["privacy_info"] != "{}"]
df.reset_index(drop=True, inplace=True)

In [ ]:
import time
def process_query(query):
    state = State(query=query)
    try:
        result = graph.invoke(state)
        return result["response"]
    except:
        for _ in range(5):
            time.sleep(5)
            try:
                result = graph.invoke(state)
                return result["response"]
            except:
                raise Exception("Failed to generate response")
    return graph.invoke(state)["response"]

In [ ]:
# Create a response column if it doesn't exist
if "response" not in df.columns:
    df["response"] = None

batch_size = 10

if os.path.exists(response_file_path):
    df = pd.read_csv(response_file_path)

In [ ]:
# Process each attack prompt and store the response
for i in tqdm(range(len(df))):
    if not pd.isna(df.loc[i, "response"]):
        continue
    df.loc[i, "response"] = process_query(df["attack_prompt"][i])
    if i % batch_size == 0 or i == len(df) - 1:
        df.to_csv(response_file_path, index=False)

In [ ]:
# 3. Evaluation

In [ ]:
from ragpas.privacy import calculateTargetIdentification

In [ ]:
top_k = 5
# 初始化累加器和计数器
total_score = 0.0
valid_count = 0
dataset_file_path = f"{root_dir}/data/output-doubao-1-5-pro/dataset-2000_TI_top{top_k}.csv"

In [ ]:
batch_size = 10

dataset = pd.DataFrame(columns=["original_context", "known_info", "attack_prompt", "response", "score", "context", "target", "privacy_info"]).to_csv(dataset_file_path, index=False)

batch_original_context = []
batch_known_info = []
batch_attack_prompts = []
batch_responses = []
batch_scores = []
batch_contexts = []
batch_targets = []
batch_privacy_info = []


df = pd.read_csv(response_file_path)

In [ ]:
for i, row in tqdm(enumerate(df.itertuples()), total=len(df)):
    if i < 600:
        continue
    try:
        score = calculateTargetIdentification(
            response=row.response,
            known_info=row.known_info,
            original_context=row.original_context,
            collection_name="retrieval_database",
            top_k = top_k
        )
        if score < 0:
            continue
    except Exception as e:
        print(f"Error in row {i}: {e}")
        continue

    # 累加总得分和计数
    total_score += score
    valid_count += 1

    batch_original_context.append(row.original_context)
    batch_known_info.append(row.known_info)
    batch_attack_prompts.append(row.attack_prompt)
    batch_responses.append(row.response)
    batch_scores.append(score)
    batch_contexts.append(row.context)
    batch_targets.append(row.target)
    batch_privacy_info.append(row.privacy_info)


    if (i + 1) % batch_size == 0 or i == len(df) - 1:
        batch = pd.DataFrame({
            "original_context": batch_original_context,
            "known_info": batch_known_info,
            "attack_prompt": batch_attack_prompts,
            "response": batch_responses,
            "score": batch_scores,
            "context": batch_contexts,
            "target": batch_targets,
            "privacy_info": batch_privacy_info
        })
        batch.to_csv(dataset_file_path, mode="a", header=False, index=False)
        batch_original_context = []
        batch_known_info = []
        batch_attack_prompts = []
        batch_responses = []
        batch_scores = []
        batch_contexts = []
        batch_targets = []
        batch_privacy_info = []

In [ ]:
# 计算平均得分
if valid_count > 0:
    average_score = total_score / valid_count
    print(f"Average Score: {average_score:.4f}")
else:
    print("No valid scores to calculate average.")